In [2]:
# !pip install pennylane

In [3]:
import numpy as np
import pennylane as qml
import torch

import pandas as pd
from torch.utils.data import DataLoader

In [4]:
# qml.about()

In [5]:
# Functions for circuit building (can skip)
def data_shape_transform(data, n_qubits=8):
  # data = data.detach().numpy()
  data = torch.reshape(data, (-1, 1))
  out = []
  col_num = int(np.ceil(len(data)/n_qubits))
  for i in range(n_qubits):
    tmp = []
    for j in range(col_num):
      if i*col_num+j >= len(data):
        break

      tmp.append(data[i*col_num+j])

    out.append(tmp)

  # print(out)
  return out, len(data)

def data_embedding(data, n_qubits=8):
  data, data_len = data_shape_transform(data, n_qubits=n_qubits)

  flag = 1
  col_num = int(np.ceil(data_len/n_qubits))
  qml.broadcast(qml.Hadamard, wires=range(n_qubits), pattern="single")

  for j in range(col_num):
    for i in range(n_qubits):
      if i*col_num+j >= data_len:
        break

      if flag == 1:
        qml.RZ(data[i][j], wires=i)
      else:
        qml.RX(data[i][j], wires=i)

    flag *= -1


def apply_weights(weights, n_qubits=8):
  for i in range(len(weights)):

    for j in range(n_qubits):
      qml.RZ(weights[i][j], wires=j)

    index = j
    flag = 1
    for k in range(n_qubits):
      for j in range(n_qubits):
        if k == j:
          continue
        index += 1
        if flag == 1:
          qml.CRX(weights[i][index], wires=[k, j])
        else:
          qml.CRY(weights[i][index], wires=[k, j])

      flag *= -1

    for j in range(n_qubits):
      qml.RZ(weights[i][index+j], wires=j)

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, X in enumerate(dataloader):
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, X)

        loss.backward()
        optimizer.step()

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, X).item()

    test_loss /= num_batches
    print(f"Avg loss: {test_loss:>8f} \n")
    return test_loss

def train(model, train_dataloader, epochs, loss_fn, optimizer):
    accuracy_hist = []
    loss_hist = []

    for epoch in range(epochs):
        train_loop(train_dataloader, model, loss_fn, optimizer)
        if epoch % 1 == 0:
          print(f"Epoch {epoch+1}\n-------------------------------")
          loss = test_loop(train_dataloader, model, loss_fn)
          loss_hist.append(loss)

    print("Finish")
    return accuracy_hist, loss_hist

In [7]:
n_qubits = 8
dev = qml.device("default.qubit", wires=n_qubits)

In [8]:
@qml.qnode(dev, interface="torch")
def c1(inputs, weights):
  data_embedding(inputs, n_qubits=8)
  apply_weights(weights, n_qubits=8)

  return qml.probs(wires=range(n_qubits))

In [9]:
class Q1Layer(torch.nn.Module):

    def __init__(self, circuit=None, n_qubits=8, q_depth=1):
        super().__init__()

        n_args = (q_depth, n_qubits**2 + n_qubits)
        weight_shapes = {"weights": n_args}
        self.qlayer_1 = qml.qnn.TorchLayer(circuit, weight_shapes)

    def forward(self, input_):
        q = self.qlayer_1(inputs=input_[0])
        q = q.unsqueeze(0)
        q = torch.reshape(q, (1, 16, 16))
        return q

In [23]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size = 3, stride=1, padding="valid"),
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels = 16, out_channels = 16, kernel_size = 3, stride=1, padding="valid"),
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels = 16, out_channels = 8, kernel_size = 5, stride=1, padding="valid"),
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels = 8, out_channels = 5, kernel_size = 3, stride=1, padding="valid"),
    torch.nn.Sigmoid(),
    torch.nn.Flatten(),

    Q1Layer(circuit=c1, n_qubits=8, q_depth=3),

    torch.nn.Conv2d(in_channels = 1, out_channels = 5, kernel_size = 3, stride=1, padding="same"),
    torch.nn.ReLU(),
    torch.nn.Conv2d(in_channels = 5, out_channels = 1, kernel_size = 3, stride=1, padding="same"),
    torch.nn.Sigmoid()
)

# model = model.double()

# sp = torch.nn.Flatten()
# l1 = torch.nn.Linear(256, 180)
# f1 = torch.nn.Sigmoid()
# ql = Q1Layer(circuit=c1, n_qubits=8, q_depth=1)

# model = torch.nn.Sequential(sp, l1, f1, ql)
# model.double()

In [24]:
for param in model.parameters():
  param.requires_grad = True

In [25]:
data = pd.read_excel('prepared_dataset.xlsx')
data = pd.DataFrame(data).to_numpy()

data = np.reshape(data, (len(data), 16, 16))

In [26]:
batch_size = 1

data = torch.tensor(data, requires_grad=True).float()

# data = torch.from_numpy(data)
# train_dataloader = DataLoader(torch.from_numpy(data), batch_size=batch_size)

In [27]:
# print(data.dtype)
# data = data.to(torch.float)
# print(data.dtype)

In [28]:
train_dataloader = DataLoader(data, batch_size=batch_size)

In [29]:
lr = 0.1
epochs = 40

In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = torch.nn.L1Loss()

In [31]:
print(model[6].weight[0])
print(model[10].weight[0])
before_training_first = model[0].weight
before_training_last = model[10].weight

tensor([[[-0.0300,  0.0243,  0.0611],
         [-0.0146,  0.0614, -0.0669],
         [ 0.0191,  0.0824,  0.0196]],

        [[-0.0410, -0.1106, -0.0152],
         [ 0.0852,  0.0899, -0.0119],
         [ 0.0423, -0.0329,  0.0295]],

        [[ 0.1120,  0.0156, -0.0653],
         [-0.0813,  0.0295,  0.1063],
         [ 0.0867,  0.0552, -0.0338]],

        [[-0.0259,  0.0162,  0.0290],
         [ 0.0230, -0.1061, -0.1136],
         [-0.0336,  0.0987,  0.0722]],

        [[-0.0827,  0.0437, -0.1094],
         [-0.1069, -0.0284, -0.0063],
         [-0.1106,  0.0981, -0.0094]],

        [[ 0.0711, -0.0435, -0.0144],
         [-0.0158,  0.0510,  0.0587],
         [-0.0635,  0.0498,  0.0874]],

        [[-0.0742, -0.0307,  0.0846],
         [-0.1017, -0.0080, -0.1011],
         [-0.1013,  0.0491, -0.0098]],

        [[ 0.0711,  0.0062, -0.0199],
         [-0.0842, -0.0533,  0.0265],
         [ 0.0129, -0.0614, -0.0996]]], grad_fn=<SelectBackward0>)
tensor([[[-0.0479, -0.2080,  0.3233],
       

In [32]:
for epoch in range(epochs):

    running_loss = 0
    size = len(train_dataloader)

    for x in train_dataloader:
        optimizer.zero_grad()

        loss_evaluated = loss_fn(model(x), x)
        loss_evaluated.backward()

        optimizer.step()

        running_loss += loss_evaluated

    avg_loss = running_loss/size
    print("Average loss over epoch {}: {:.4f}".format(epoch + 1, avg_loss))

Average loss over epoch 1: 0.3186
Average loss over epoch 2: 0.2641
Average loss over epoch 3: 0.2627
Average loss over epoch 4: 0.2627
Average loss over epoch 5: 0.2627
Average loss over epoch 6: 0.2627
Average loss over epoch 7: 0.2627
Average loss over epoch 8: 0.2627
Average loss over epoch 9: 0.2627
Average loss over epoch 10: 0.2627
Average loss over epoch 11: 0.2627
Average loss over epoch 12: 0.2627
Average loss over epoch 13: 0.2627
Average loss over epoch 14: 0.2627
Average loss over epoch 15: 0.2627
Average loss over epoch 16: 0.2627
Average loss over epoch 17: 0.2627
Average loss over epoch 18: 0.2627
Average loss over epoch 19: 0.2627
Average loss over epoch 20: 0.2627
Average loss over epoch 21: 0.2627
Average loss over epoch 22: 0.2627
Average loss over epoch 23: 0.2627
Average loss over epoch 24: 0.2627
Average loss over epoch 25: 0.2627
Average loss over epoch 26: 0.2627
Average loss over epoch 27: 0.2627
Average loss over epoch 28: 0.2627
Average loss over epoch 29: 0

In [33]:
test_loop(train_dataloader, model, loss_fn)

Avg loss: 0.262657 



0.2626572012901306

In [34]:
print(model[0].weight[0:3])
print(model[10].weight[0:3])
after_training_first = model[0].weight
after_training_last = model[10].weight

tensor([[[[-0.3716, -0.4663, -0.4798],
          [-0.0973, -0.1909, -0.6888],
          [-0.5084, -0.2886, -0.5322]]],


        [[[ 0.0921,  0.6459,  0.4353],
          [-0.3653, -0.2966, -0.2284],
          [-0.4189, -0.4472, -0.6616]]],


        [[[-0.0875,  0.4398, -0.4813],
          [-0.5354, -0.3517, -0.4402],
          [-0.2684, -0.2001, -0.4975]]]], grad_fn=<SliceBackward0>)
tensor([[[[ 1.2003,  0.8362,  1.4726],
          [ 1.2676,  1.4912,  1.1225],
          [ 1.1384,  1.1684,  1.3969]]],


        [[[ 1.0813,  1.3246,  0.8870],
          [ 1.0634,  1.1300,  0.8639],
          [ 0.6684,  0.8387,  1.0243]]],


        [[[-0.5897, -0.8066, -0.4446],
          [-0.3155, -0.3791, -0.3083],
          [-0.7911, -0.4665, -0.7870]]]], grad_fn=<SliceBackward0>)
